In [ ]:
ALTER SESSION SET TIMEZONE = 'Europe/London';

In [ ]:
try:
    import requests
    import pandas as pd
    from requests.auth import HTTPBasicAuth 
    from snowflake.snowpark.context import get_active_session
    import _snowflake
    session = get_active_session()
    personal_access_token = session.sql("SELECT prod.raw.ado_secrets()").collect()[0][0]

except:    
    import requests
    import pandas as pd
    from requests.auth import HTTPBasicAuth 
    from snowflake.snowpark.context import get_active_session


In [ ]:
class Azure_DevOps:
    def __init__(self):
        self.organization_name = "Phlexglobal"
        self.personal_access_token = personal_access_token
        self.auth = HTTPBasicAuth('', self.personal_access_token)
        self.params = {'$top': 5000}
        self.pipelines = [2441]
        
        # to be changed into secret
        self.sub_id = '8642d1ba-f201-45c8-955d-2347876e4145'
    
    def get_ado_projects(self): 
        url_projects = f"https://dev.azure.com/Phlexglobal/_apis/projects?api-version=7.1"
        
        response = requests.get(url_projects, auth=self.auth, params = self.params)
        
        resp = response.json()['value']
        df_projects = pd.DataFrame(resp)
        unique_ids = df_projects['id'] .astype(str).unique().tolist()
        
        new_order = ['id', 'url', 'name', 'state', 'revision', 'visibility', 'description', 'lastUpdateTime']
        df_projects = df_projects[new_order]
        
        df_projects.columns = ['ID', 'URL', 'NAME', 'STATE', 'REVISION', 'VISIBILITY',  'DESCRIPTION', 'LASTUPDATETIME']
        return df_projects


    def get_unique_projects(self):
        unique_ids = self.get_ado_projects()['ID'] .astype(str).unique().tolist()
        return unique_ids
        
    def get_ado_approvals(self): 
    
        url_projects = f"https://dev.azure.com/Phlexglobal/_apis/projects?api-version=7.1"  
    
        approvals = []
    
        for i in self.get_unique_projects():
            url_approvals = f"https://dev.azure.com/Phlexglobal/{i}/_apis/pipelines/approvals?api-version=7.1"
            response = requests.get(url_approvals, auth=self.auth, params = self.params)
            resp = response.json()['value']
            approvals.extend(resp)
    
        df_approvals = pd.DataFrame(approvals)
    
    
        new_order = ['id', 'steps', '_links', 'status', 'pipeline', 'createdOn', 'executionOrder', 'lastModifiedOn', 'blockedApprovers', 'minRequiredApprovers']
        df_approvals = df_approvals[new_order]
    
        df_approvals.columns = ['ID', 'STEPS', '_LINKS', 'STATUS', 'PIPELINE', 'CREATEDON', 'EXECUTIONORDER', 'LASTMODIFIEDON', 'BLOCKEDAPPROVERS', 'MINREQUIREDAPPROVERS']
    
        return df_approvals

    def get_ado_definitions(self): 
    
        definitions = []
    
        for i in self.get_unique_projects():
    
            url_definitions = f"https://dev.azure.com/Phlexglobal/{i}/_apis/build/definitions?api-version=7.1"
            response = requests.get(url_definitions, auth=self.auth, params = self.params)
            resp = response.json()['value']
            definitions.extend(resp)
            
        df_definitions = pd.DataFrame(definitions)
    
    
        new_order = [
            'id', 
            'uri', 
            'url', 
            'name', 
            'path', 
            'type', 
            'queue', 
            '_links', 
            'drafts', 
            'project', 
            'quality', 
            'revision', 
            'authoredBy', 
            'createdDate', 
            'queueStatus'
        ]
    
        df_definitions = df_definitions[new_order]
    
        df_definitions.columns = [
            'ID', 
            'URI', 
            'URL', 
            'NAME', 
            'PATH', 
            'TYPE', 
            'QUEUE', 
            '_LINKS', 
            'DRAFTS', 
            'PROJECT', 
            'QUALITY', 
            'REVISION', 
            'AUTHOREDBY', 
            'CREATEDDATE', 
            'QUEUESTATUS'
        ]
    
        df_definitions['PROJECTID'] = df_definitions['PROJECT'].apply(lambda x: x['id'])
        df_definitions['PROJECTNAME'] = df_definitions['PROJECT'].apply(lambda x: x['name'])
    
        return df_definitions
        
    def get_ado_pipelines(self):

        pipelines = []
        for i in self.get_unique_projects():
            url_pipelines = f"https://dev.azure.com/Phlexglobal/{i}/_apis/pipelines?api-version=7.1"
            response = requests.get(url_pipelines, auth=self.auth, params=self.params)
            resp = response.json()['value']
            pipelines.extend(resp)
            
        df_pipelines = pd.DataFrame(pipelines)
    
        new_order = [
            'id', 
            'url', 
            'name', 
            '_links', 
            'folder', 
            'revision'
        ]
    
        df_pipelines = df_pipelines[new_order]
    
        df_pipelines.columns = [
            'ID', 
            'URL', 
            'NAME', 
            '_LINKS', 
            'FOLDER', 
            'REVISION'
        ]
        return df_pipelines

        
    def get_ado_source_providers(self):

        src_providers = []
        for i in self.get_unique_projects():
            url_src_providers = f"https://dev.azure.com/Phlexglobal/{i}/_apis/sourceproviders?api-version=7.1"
            response = requests.get(url_src_providers, auth=self.auth, params=self.params)
            resp = response.json()['value']
            src_providers.extend(resp)
        df_source_providers = pd.DataFrame(src_providers)
    
    
        new_order = [
            'name', 
            'supportedTriggers', 
            'supportedCapabilities'
        ]
    
        df_source_providers = df_source_providers[new_order]
    
        df_source_providers.columns = [
            'NAME', 
            'SUPPORTEDTRIGGERS', 
            'SUPPORTEDCAPABILITIES'
        ]   
        return df_source_providers

    def get_ado_repos(self):

        repos = []
        for i in self.get_unique_projects():
            url_repos =f"https://dev.azure.com/PhlexGlobal/{i}/_apis/git/repositories?api-version=4.1"
            response = requests.get(url_repos,auth=self.auth, params=self.params)
            resp = response.json()['value']
            repos.extend(resp)
        df_repos = pd.DataFrame(repos)
    
        new_order = [
            'id', 
            'url', 
            'name', 
            'size', 
            'sshUrl', 
            'webUrl', 
            'project', 
            'remoteUrl', 
            'isDisabled', 
            'defaultBranch', 
            'isInMaintenance'
        ]
    
        df_repos = df_repos[new_order]
    
    
        df_repos.columns = [
            'ID', 
            'URL', 
            'NAME', 
            'SIZE', 
            'SSHURL', 
            'WEBURL', 
            'PROJECT', 
            'REMOTEURL', 
            'ISDISABLED', 
            'DEFAULTBRANCH', 
            'ISINMAINTENANCE'
        ]
    
        df_repos['proj_id'] = df_repos['PROJECT'].apply(lambda x: x['id'])
    
        df_repos_uniques = df_repos['ID'].astype(str).unique().tolist()
    
        return df_repos

    def get_unique_pipeline_runs(self):
      
        for i in self.pipelines:
            url = f"https://dev.azure.com/Phlexglobal/{self.sub_id}/_apis/pipelines/{i}/runs?api-version=7.1"
            response = requests.get(url, auth=self.auth, params=self.params)
        
            resp = response.json()
        
            df = pd.DataFrame(resp['value']) 
        
            # Filter by name containing 'PEL'
            filtered_df = df[df['name'].str.contains('PEL', case=False, na=False)].copy()
            filtered_df['issue_key'] = filtered_df['name'].astype(str).str.split('#').str[1]
            filtered_df['env'] = filtered_df['name'].astype(str).str.split('#').str[-1]
            unique_names = filtered_df['issue_key'].unique().tolist()
            
        return unique_names, filtered_df
            
    def get_pipeline_runs(self):
        runs = []
        
        unique_names, filtered_df = self.get_unique_pipeline_runs()
        
        for i in unique_names:  # Now iterate over unique_names
            name_df = filtered_df[filtered_df['issue_key'] == i].copy()
            
            name_df['rank'] = (name_df.groupby('state')['finishedDate']
                                .rank(method='min', ascending=True))
        
            succeeded_df = name_df[name_df['result'] == 'succeeded']
            if not succeeded_df.empty:
                min_succeeded_rank = succeeded_df['rank'].min()
                name_df['first_is_succeeded'] = (
                        (name_df['result'] == 'succeeded') & 
                        (name_df['rank'] == min_succeeded_rank)
                    )
            else:
                    name_df['first_is_succeeded'] = False
                
            runs.append(name_df)
        
        # Move this outside the loop
        final_df = pd.concat(runs, ignore_index=True).drop(columns='templateParameters')
        return final_df
        
api = Azure_DevOps()
        

In [ ]:
df_projects = api.get_ado_projects()
df_projects

In [ ]:
temp_projects_table = 'PROJECTS_FROM_PYTHON'

unique_ids = df_projects['ID'] .astype(str).unique().tolist()

session.write_pandas(df_projects,
                         table_name=temp_projects_table,
                         auto_create_table=True,
                         overwrite=True,
                         table_type="temporary")

In [ ]:
create or replace table prod.raw.ado_projects
(
ID string,
URL string,
NAME string,
STATE string,
REVISION float,
VISIBILITY string,
DESCRIPTION string,
LASTUPDATETIME timestamp_tz,
timestamp timestamp_tz
);

truncate table prod.raw.ado_projects;

INSERT INTO prod.raw.ado_projects
SELECT
    "ID",
    "URL",
    "NAME",
    "STATE",
    "REVISION",
    "VISIBILITY",
    "DESCRIPTION",
    "LASTUPDATETIME",
    current_timestamp as timestamp
FROM PROJECTS_FROM_PYTHON;

select top 15 * from prod.raw.ado_projects

In [ ]:
-- select * from prod.raw.ado_projects

In [ ]:
approvals = api.get_ado_approvals()
approvals

In [ ]:
temp_approvals_table = 'TEMP_APPROVALS'

session.write_pandas(approvals,
                         table_name=temp_approvals_table,
                         auto_create_table=True,
                         overwrite=True,
                         table_type="temporary")

In [ ]:
-- CREATE OR REPLACE TABLE prod.raw.ado_approvals (
--     ID VARCHAR(16777216),
--     STEPS ARRAY,
--     _LINKS OBJECT,
--     STATUS VARCHAR(16777216),
--     PIPELINE OBJECT,
--     CREATEDON VARCHAR(16777216),
--     EXECUTIONORDER VARCHAR(16777216),
--     LASTMODIFIEDON VARCHAR(16777216),
--     BLOCKEDAPPROVERS ARRAY,
--     MINREQUIREDAPPROVERS FLOAT,
--     PIPELINEID int,
--     PIPELINEOWNERID int,
--     PIPELINENAME string,
--     timestamp timestamp_tz
-- );

TRUNCATE TABLE prod.raw.ado_approvals;

INSERT INTO prod.raw.ado_approvals
SELECT
    ID,
    STEPS,
    _LINKS,
    STATUS,
    PIPELINE,
    CREATEDON,
    EXECUTIONORDER,
    LASTMODIFIEDON,
    BLOCKEDAPPROVERS,
    MINREQUIREDAPPROVERS,
    parse_json(PIPELINE):id::int as PIPELINEID,
    parse_json(PIPELINE):owner.id::int as PIPELINEOWNERID,
    parse_json(PIPELINE):name::string as PIPELINENAME,
    current_timestamp as timestamp
FROM TEMP_APPROVALS;

SELECT TOP 15 * FROM prod.raw.ado_approvals;

In [ ]:
CREATE OR REPLACE TABLE prod.raw.ADO_BUILD_SNOWFLAKE_TMP AS
WITH b AS (
    -- Get all project IDs and names
    SELECT
        PROJECT_NAME,
        REPO_ID,
        prod.raw.get_builds(PROJECT_NAME, REPO_ID) AS builds,
        CURRENT_TIMESTAMP AS timestamp
    FROM prod.raw.ado_repos
),
flattened_data AS (
    SELECT
        b.PROJECT_NAME::VARIANT as PROJECT_NAME,
        b.REPO_ID::VARIANT as REPO_ID,
        b.timestamp,
        f.value:URI::STRING AS URI,
        f.value:URL::STRING AS URL,
        f.value:LOGS::STRING AS LOGS,
        f.value:TAGS::ARRAY AS TAGS,
        f.value:PLANS::ARRAY AS PLANS,
        f.value:QUEUE::OBJECT AS QUEUE,
        f.value:_LINKS::OBJECT AS _LINKS,
        f.value:REASON::STRING AS REASON,
        f.value:RESULT::STRING AS RESULT,
        f.value:STATUS::STRING AS STATUS,
        f.value:PROJECT::OBJECT AS PROJECT,
        f.value:PRIORITY::STRING AS PRIORITY,
        f.value:QUEUETIME::STRING AS QUEUETIME,
        f.value:STARTTIME::STRING AS STARTTIME,
        f.value:DEFINITION::OBJECT AS DEFINITION,
        f.value:FINISHTIME::STRING AS FINISHTIME,
        f.value:PARAMETERS::STRING AS PARAMETERS,
        f.value:PROPERTIES::OBJECT AS PROPERTIES,
        f.value:REPOSITORY::OBJECT AS REPOSITORY,
        f.value:BUILDNUMBER::STRING AS BUILDNUMBER,
        f.value:REQUESTEDBY::OBJECT AS REQUESTEDBY,
        f.value:TRIGGERINFO::OBJECT AS TRIGGERINFO,
        f.value:REQUESTEDFOR::OBJECT AS REQUESTEDFOR,
        f.value:SOURCEBRANCH::STRING AS SOURCEBRANCH,
        f.value:LASTCHANGEDBY::OBJECT AS LASTCHANGEDBY,
        f.value:SOURCEVERSION::STRING AS SOURCEVERSION,
        f.value:LASTCHANGEDDATE::STRING AS LASTCHANGEDDATE,
        f.value:ORCHESTRATIONPLAN::OBJECT AS ORCHESTRATIONPLAN,
        f.value:RETAINEDBYRELEASE::BOOLEAN AS RETAINEDBYRELEASE,
        f.value:VALIDATIONRESULTS::ARRAY AS VALIDATIONRESULTS,
        f.value:TEMPLATEPARAMETERS::OBJECT AS TEMPLATEPARAMETERS,
        f.value:BUILDNUMBERREVISION::FLOAT AS BUILDNUMBERREVISION,
        f.value:APPENDCOMMITMESSAGETORUNNAME::BOOLEAN AS APPENDCOMMITMESSAGETORUNNAME
    FROM b,
    LATERAL FLATTEN(INPUT => PARSE_JSON(b.builds):builds) f
)
SELECT * FROM flattened_data;

In [ ]:
INSERT INTO prod.raw.ADO_BUILD_SNOWFLAKE
SELECT DISTINCT *
FROM prod.raw.ADO_BUILD_SNOWFLAKE_TMP
WHERE NOT EXISTS (
    SELECT 1 
    FROM prod.raw.ADO_BUILD_SNOWFLAKE target
    WHERE target.URI = prod.raw.ADO_BUILD_SNOWFLAKE_TMP.URI
);

In [ ]:
CREATE OR REPLACE  TABLE prod.silver.ADO_BUILDS_SNOWFLAKE
as
SELECT
    reason,
    result, 
    status, 
    project:id::string AS projectId,
    priority,
    CAST(queuetime AS DATETIME) AS queuetime,
    CAST(starttime AS DATETIME) AS starttime,
    CAST(finishtime AS DATETIME) AS finishtime,
    definition:id::string AS definitionId,
    repository:id::string AS repositoryId,
    buildnumber,
    requestedby:displayName::string AS requestedBy,
    requestedfor:displayName::string AS requestedFor,
    lastchangedby:displayName::string AS lastChangedBy,
    sourcebranch,
    CAST(lastchangeddate AS DATETIME) AS lastchangeddate,
    buildnumberrevision,
    SPLIT_PART(URI, '/', -1) AS BuldId,
    current_timestamp as timestamp

FROM prod.raw.ADO_BUILD_SNOWFLAKE;

In [ ]:
definitions = api.get_ado_definitions()
definitions

In [ ]:
temp_definitions_table = 'TEMP_DEFINITIONS'

session.write_pandas(definitions,
                         table_name=temp_definitions_table,
                         auto_create_table=True,
                         overwrite=True,
                         table_type="temporary")

In [ ]:
-- CREATE OR REPLACE TABLE prod.raw.ado_definitions (
--     ID FLOAT,
--     URI VARCHAR(16777216),
--     URL VARCHAR(16777216),
--     NAME VARCHAR(16777216),
--     PATH VARCHAR(16777216),
--     TYPE VARCHAR(16777216),
--     QUEUE OBJECT,
--     _LINKS OBJECT,
--     DRAFTS ARRAY,
--     PROJECT OBJECT,
--     QUALITY VARCHAR(16777216),
--     REVISION FLOAT,
--     AUTHOREDBY OBJECT,
--     CREATEDDATE VARCHAR(16777216),
--     QUEUESTATUS VARCHAR(16777216),
--     PROJECTID string,
--     PROJECTNAME string,
--     timestamp timestamp_tz
-- );

TRUNCATE TABLE prod.raw.ado_definitions;

INSERT INTO prod.raw.ado_definitions
SELECT
    "ID",
    "URI",
    "URL",
    "NAME",
    "PATH",
    "TYPE",
    "QUEUE",
    "_LINKS",
    "DRAFTS",
    "PROJECT",
    "QUALITY",
    "REVISION",
    "AUTHOREDBY",
    "CREATEDDATE",
    "QUEUESTATUS",
    "PROJECTID",
    "PROJECTNAME",
    current_timestamp as timestamp
FROM TEMP_DEFINITIONS;

SELECT TOP 15 * FROM prod.raw.ado_definitions;

In [ ]:
pipelines = api.get_ado_pipelines()
pipelines

In [ ]:
temp_pipelines_table = 'TEMP_PIPELINES'
session.write_pandas(pipelines,
                         table_name=temp_pipelines_table,
                         auto_create_table=True,
                         overwrite=True,
                         table_type="temporary")

In [ ]:
-- CREATE OR REPLACE TABLE prod.raw.ado_pipelines (
--     ID FLOAT,
--     URL VARCHAR(16777216),
--     NAME VARCHAR(16777216),
--     _LINKS OBJECT,
--     FOLDER VARCHAR(16777216),
--     REVISION FLOAT,
--     timestamp timestamp_tz
-- );

TRUNCATE TABLE prod.raw.ado_pipelines;

INSERT INTO prod.raw.ado_pipelines
SELECT
    "ID",
    "URL",
    "NAME",
    "_LINKS",
    "FOLDER",
    "REVISION",
        current_timestamp as timestamp
FROM TEMP_PIPELINES;

SELECT TOP 15 * FROM prod.raw.ado_pipelines;

In [ ]:
CREATE OR REPLACE TABLE prod.raw.ADO_PULL_REQUESTS_SNOWFLAKE_TMP_ARRAY AS
WITH pull_request AS (
    SELECT
        b.PROJECT_NAME,
        b.REPO_ID,
        prod.raw.get_pull_request(b.PROJECT_NAME, b.REPO_ID) AS pull_request_data,
        CURRENT_TIMESTAMP() as timestamp
    FROM prod.raw.ado_repos b
),
split_data AS (
    SELECT
        PROJECT_NAME,
        REPO_ID,
        pull_request_data:count::INT AS count, -- Extract the count field
        pull_request_data:pullRequests AS pullRequests, -- Extract the pullRequests array
        timestamp
    FROM pull_request
)
SELECT 
    PROJECT_NAME,
    REPO_ID,
    count,
    pullRequests,
    timestamp
FROM split_data
WHERE count > 0;

select top 10 * from prod.raw.ADO_PULL_REQUESTS_SNOWFLAKE_TMP_ARRAY

In [ ]:
CREATE OR REPLACE TABLE prod.raw.ADO_PULL_REQUESTS_SNOWFLAKE_TMP AS
WITH FlattenedData AS (
   SELECT 
        PROJECT_NAME,
        REPO_ID,
        timestamp,
        value:repository AS repository,
        value:pullRequestId::STRING AS pullRequestId,
        value:codeReviewId::STRING AS codeReviewId,
        value:status::STRING AS status,
        value:createdBy AS createdBy,
        value:creationDate::STRING AS creationDate,
        value:closedDate::STRING AS closedDate,
        value:title::STRING AS title,
        value:description::STRING AS description,
        value:sourceRefName::STRING AS sourceRefName,
        value:targetRefName::STRING AS targetRefName,
        value:mergeStatus::STRING AS mergeStatus,
        value:isDraft::BOOLEAN AS isDraft,
        value:mergeId::STRING AS mergeId,
        value:lastMergeSourceCommit AS lastMergeSourceCommit,
        value:lastMergeTargetCommit AS lastMergeTargetCommit,
        value:lastMergeCommit AS lastMergeCommit,
        value:reviewers AS reviewers,
        value:url::STRING AS url,
        value:completionOptions AS completionOptions,
        value:supportsIterations::BOOLEAN AS supportsIterations
    FROM prod.raw.ADO_PULL_REQUESTS_SNOWFLAKE_TMP_ARRAY,
    LATERAL FLATTEN(input => pullRequests) -- Replace with your array column name
)
SELECT *
FROM FlattenedData;

select top 10 * from prod.raw.ADO_PULL_REQUESTS_SNOWFLAKE_TMP

In [ ]:
CREATE OR REPLACE TABLE prod.raw.ADO_PULL_REQUESTS_SNOWFLAKE AS
SELECT  *
FROM prod.raw.ADO_PULL_REQUESTS_SNOWFLAKE_TMP;

In [ ]:
INSERT INTO prod.raw.ADO_PULL_REQUESTS_SNOWFLAKE
SELECT DISTINCT *
FROM prod.raw.ADO_PULL_REQUESTS_SNOWFLAKE_TMP
WHERE NOT EXISTS (
    SELECT 1 
    FROM prod.raw.ADO_PULL_REQUESTS_SNOWFLAKE target
    WHERE target.pullRequestId = ADO_PULL_REQUESTS_SNOWFLAKE_TMP.pullRequestId
);

In [ ]:
CREATE OR REPLACE TABLE prod.silver.ADO_PULL_REQUESTS_SNOWFLAKE
AS
WITH base_data AS (
    SELECT
        pullRequestId,
        mergeId,
        title,
        description,
        status,
        isDraft,
        CAST(creationDate AS DATETIME) AS creationDate,
        CAST(closedDate AS DATETIME) AS closedDate,
        DATEDIFF(MINUTE, creationDate, closedDate) AS age,
        mergeStatus,
        PARSE_JSON(createdby) AS createdByJson,
        PARSE_JSON(repository) AS repositoryJson,
        timestamp
--        reviewers
    FROM prod.raw.ADO_PULL_REQUESTS_SNOWFLAKE
)

SELECT
    pullRequestId,
    mergeId,
    title,
    description,
    status,
    isDraft,
    creationDate,
    closedDate,
    age,
    mergeStatus,
    createdByJson:displayName::STRING AS createdBy,
--    reviewer.value:displayName::STRING AS reviewers,
    repositoryJson:"name"::STRING AS repository,
    repositoryJson:"id"::STRING AS repositoryId,
    timestamp
FROM base_data;
--   LATERAL FLATTEN(input => base_data.reviewers) AS reviewer;

In [ ]:
-- select * from prod.silver.ADO_PULL_REQUESTS_SNOWFLAKE
-- where closeddate is null

In [ ]:
drop table prod.raw.ADO_PULL_REQUESTS_SNOWFLAKE_TMP;
drop table prod.raw.ADO_PULL_REQUESTS_SNOWFLAKE_TMP_ARRAY;

In [ ]:
source_providers = api.get_ado_source_providers()
source_providers

In [ ]:

temp_pull_requests_table = 'TEMP_SOURCE_PROVIDERS'
session.write_pandas(source_providers,
                         table_name=temp_pull_requests_table,
                         auto_create_table=True,
                         overwrite=True,
                         table_type="temporary")

In [ ]:
CREATE OR REPLACE TABLE prod.raw.ado_source_providers (
    NAME VARCHAR(16777216),
    SUPPORTEDTRIGGERS ARRAY,
    SUPPORTEDCAPABILITIES OBJECT,
    timestamp timestamp_tz
);

TRUNCATE TABLE prod.raw.ado_source_providers;

INSERT INTO prod.raw.ado_source_providers
SELECT
    "NAME",
    "SUPPORTEDTRIGGERS",
    "SUPPORTEDCAPABILITIES",
        current_timestamp as timestamp
FROM TEMP_SOURCE_PROVIDERS;

SELECT TOP 15 * FROM prod.raw.ado_source_providers;

In [ ]:
repos = api.get_ado_repos()
repos

In [ ]:
temp_repos_table = 'TEMP_REPOS'
session.write_pandas(repos,
                         table_name=temp_repos_table,
                         auto_create_table=True,
                         overwrite=True,
                         table_type="temporary")



In [ ]:
-- CREATE OR REPLACE TABLE prod.raw.ado_repos (
--     REPO_ID VARCHAR(16777216),
--     URL VARCHAR(16777216),
--     NAME VARCHAR(16777216),
--     SIZE FLOAT,
--     SSHURL VARCHAR(16777216),
--     WEBURL VARCHAR(16777216),
--     PROJECT VARCHAR(16777216),
--     REMOTEURL VARCHAR(16777216),
--     ISDISABLED BOOLEAN,
--     DEFAULTBRANCH VARCHAR(16777216),
--     ISINMAINTENANCE BOOLEAN,
--     PROJECT_NAME string,
--     timestamp timestamp_tz
-- );

TRUNCATE TABLE prod.raw.ado_repos;

INSERT INTO prod.raw.ado_repos
SELECT
    ID as REPO_ID,
    URL,
    NAME,
    SIZE,
    SSHURL,
    WEBURL,
    PROJECT,
    REMOTEURL,
    ISDISABLED,
    DEFAULTBRANCH,
    ISINMAINTENANCE,
    parse_json(PROJECT):name::string as PROJECT_NAME,
        current_timestamp as timestamp
FROM TEMP_REPOS;

SELECT TOP 15 * FROM prod.raw.ado_repos;

In [ ]:
CREATE OR REPLACE TABLE prod.raw.ado_branches AS
WITH projects AS (
    -- Fetch project names and repository IDs
    SELECT PROJECT_NAME, REPO_ID
    FROM prod.raw.ado_repos
),
branches_data AS (
    -- Call get_branches for each project and repository
    SELECT
        p.PROJECT_NAME,
        p.REPO_ID,
        b.value AS branch_name,
            current_timestamp as timestamp
    FROM projects p,
         LATERAL FLATTEN(INPUT => prod.raw.get_branches(p.PROJECT_NAME, p.REPO_ID)) b
)
SELECT *
FROM branches_data;

In [ ]:
CREATE OR REPLACE TABLE prod.raw.ado_commits (
    REPO_ID STRING,
    BRANCH_NAME STRING,
    RESULT_COMMIT STRING,
    timestamp timestamp_tz
);

INSERT INTO prod.raw.ado_commits (REPO_ID, BRANCH_NAME, RESULT_COMMIT, timestamp)
SELECT 
    REPO_ID, 
    BRANCH_NAME,
    prod.raw.get_commits(REPO_ID, BRANCH_NAME) AS RESULT_COMMIT,
        current_timestamp as timestamp
FROM 
    prod.raw.ado_branches;

In [ ]:
pipeline_runs = api.get_pipeline_runs()
pipeline_runs

In [ ]:
temp_pipeline_runs_table = 'TEMP_PIPELINE_RUNS'
session.write_pandas(pipeline_runs,
                         table_name=temp_pipeline_runs_table,
                         auto_create_table=True,
                         overwrite=True,
                         table_type="temporary")

In [ ]:
create or replace table prod.raw.ado_pipeline_runs as
select
*
from TEMP_PIPELINE_RUNS


In [ ]:
select * from prod.raw.ado_pipeline_runs 

In [ ]:
create or replace table prod.silver.ado_pipeline_runs as
select
"env"::string as environment,
"pipeline"::variant as pipeline,
"state"::varchar as state,
"url"::varchar as url,
"id"::int as id,
"result"::varchar as result,
"createdDate"::datetime as createddate,
"finishedDate"::datetime as finishedDate,
datediff(m,"createdDate","finishedDate") as run_time_minutes,
"issue_key"::varchar as issue_key,
"rank"::int as rank_from_finished_date_asc,
"first_is_succeeded"::boolean as first_success
from prod.raw.ado_pipeline_runs;


In [ ]:
select * from prod.silver.ado_pipeline_runs

In [ ]:
create or replace temp table prod.silver.ado_pipeline_runs_sum_first_succeeded as
with min_rank as (
    select issue_key,
           min(rank_from_finished_date_asc) as rank_min
    from prod.silver.ado_pipeline_runs 
    where result = 'succeeded'
    group by issue_key
)
select 
    apr.issue_key,
    sum(apr.run_time_minutes) as total_run_time_minutes
from prod.silver.ado_pipeline_runs apr
inner join min_rank mr on apr.issue_key = mr.issue_key
where apr.rank_from_finished_date_asc <= mr.rank_min
group by apr.issue_key;

In [ ]:
select * 
  from prod.silver.ado_pipeline_runs
  where issue_key = 'PEL-23'

In [ ]:
      select issue_key,
      min(rank_from_finished_date_asc)  
      from prod.silver.ado_pipeline_runs sub 
      where result = 'succeeded'
      group by issue_key

In [ ]:

create or replace table prod.silver.ado_pipeline_runs_agg as 
with first_success_run_time as (
  select 
    issue_key,
    run_time_minutes as first_succeeded_run_time
  from prod.silver.ado_pipeline_runs
  where result = 'succeeded' 
    and rank_from_finished_date_asc = (
      select min(rank_from_finished_date_asc) 
      from prod.silver.ado_pipeline_runs sub 
      where sub.issue_key = prod.silver.ado_pipeline_runs.issue_key 
        and sub.result = 'succeeded'
    )
),
first_success as (
  select 
    issue_key,
    min(rank_from_finished_date_asc) as successful_attempt,
    avg(run_time_minutes) as avg_run_time
  from prod.silver.ado_pipeline_runs
  where result = 'succeeded'
  group by issue_key
)
select
  apr.issue_key,
  fs.successful_attempt,
  concat(round(fs.avg_run_time), ' min') as avg_run_time_for_succeeded,
  concat(round(fsrt.first_succeeded_run_time), ' min') as first_succeeded_build_time,
  sfs.total_run_time_minutes as minutes_with_first_success
from (select distinct "issue_key" as issue_key from prod.raw.ado_pipeline_runs) apr
left join first_success fs on apr.issue_key = fs.issue_key
left join first_success_run_time fsrt on apr.issue_key = fsrt.issue_key
left join prod.silver.ado_pipeline_runs_sum_first_succeeded sfs on apr.issue_key = sfs.issue_key;

select * from prod.silver.ado_pipeline_runs_agg;

In [ ]:
create or replace table prod.silver.ado_pipeline_runs_agg_minutes_env as
select
-- issue_key,
environment,
sum(run_time_minutes) as summed_run_time_minutes
from prod.silver.ado_pipeline_runs
group by 1
order by 1

In [ ]:
select * from prod.silver.ado_pipeline_runs_agg_minutes_env

In [ ]:
create or replace table prod.silver.ado_pipeline_runs_agg_minutes_issue_key as
select
issue_key,
-- environment,
sum(run_time_minutes) as summed_run_time_minutes
from prod.silver.ado_pipeline_runs
group by 1
order by 1

In [ ]:
select * from prod.silver.ado_pipeline_runs_agg_minutes_issue_key

In [ ]:
select * from prod.silver.ado_pipeline_runs
where issue_key = 'PEL-23'

In [ ]:
select * from prod.silver.ado_pipeline_runs_agg

In [ ]:
# 8642d1ba-f201-45c8-955d-2347876e4145

# 2441